# 📊 EXPORTAÇÃO DE DADOS PARA POWER BI

**Objetivo:** Preparar e gerar todos os datasets necessários para criação de dashboards de análise de crédito no Power BI.

### ⏳ Arquivos gerados
- dataset_powerbi.csv – dataset principal com probabilidades e previsões
- threshold_analysis.csv – métricas para diferentes thresholds
- roc_curve.csv – dados para curva ROC de cada modelo
- precision_recall_curve.csv – dados para curva Precision-Recall
- feature_importance_comparison.csv – importância das features comparando LR e RF
- confusion_matrix.csv – matrizes de confusão por threshold
- model_comparison.csv – comparação de métricas entre modelos
- inadimplencia_ocupacao.csv – taxa de inadimplência por ocupação
- inadimplencia_score.csv – taxa de inadimplência por faixa de score
- inadimplencia_renda.csv – taxa de inadimplência por faixa de renda

In [ ]:
# CÉLULA 1. IMPORTAÇÃO DE BIBLIOTECAS
import pandas as pd
import numpy as np
import sqlite3
import os

# ML
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

# Criar pasta de destino
os.makedirs('../powerbi', exist_ok=True)

print("📊 INICIANDO EXPORTAÇÃO...")

In [ ]:
# CÉLULA 2. CARREGANDO DADOS DO SQLITE
conn = sqlite3.connect('../data/database.db')
df = pd.read_sql("SELECT * FROM clientes", conn)
conn.close()

print(f"✓ Dados carregados: {len(df):,} registros")

# Criar feature renda_per_capita
df['renda_per_capita'] = df['renda_anual'] / (df['numero_dependentes'] + 1)
print(f"✓ Feature 'renda_per_capita' criada")

In [ ]:
# CÉLULA 3. PREPARANDO DADOS PARA MODELAGEM
target = 'inadimplente'
features = [col for col in df.columns if col not in ['inadimplente', 'id_cliente']]

X = df[features].copy()
y = df[target]

# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"✓ Split: {len(X_train):,} treino / {len(X_test):,} teste")

In [ ]:
# CÉLULA 4. ENCODING DE VARIÁVEIS CATEGÓRICAS
cat_cols = X.select_dtypes(include='object').columns
encoders = {}

for col in cat_cols:
  le = LabelEncoder()
  X_train[col] = le.fit_transform(X_train[col])
  X_test[col] = le.transform(X_test[col])
  encoders[col] = le

print(f"✓ Encoding aplicado em {len(cat_cols)} colunas")

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ Normalização aplicada")

In [ ]:
# CÉLULA 5. TREINAR LR (LOGISTIC REGRESSION)
print("\n🎯 Treinando Logistic Regression...")
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train_scaled, y_train)
y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
y_pred_lr = lr.predict(X_test_scaled)

print(f"  AUC: {roc_auc_score(y_test, y_proba_lr):.4f}")
print(f"  Recall: {recall_score(y_test, y_pred_lr):.4f}")

In [ ]:
# CÉLULA 6. TREINANDO RF (RANDOM FOREST)
print("\n🌲 Treinando Random Forest...")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)
y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]
y_pred_rf = rf.predict(X_test_scaled)

print(f"AUC: {roc_auc_score(y_test, y_proba_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}") 